In [ ]:
import empyrical as ep
data2=data["收盤價"].pct_change().dropna()
print(ep.sharpe_ratio(data2))
print(ep.sortino_ratio(data2))

In [ ]:
pip install empyrical

In [ ]:
import requests
import pandas as pd
import time

def 下載股價資訊(月份列表,股票代碼):
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            print(f"{date_str[0:6]}抓取失敗")
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(0.5)  # 避免請求過快
    df = pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
    # 處理日期：轉為西元年
    df["日期"] = df["日期"].apply(民國轉西元)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    return df
    
def 民國轉西元(date_str):
    parts = date_str.split('/')
    parts[0] = str(int(parts[0]) + 1911)
    return '/'.join(parts)
    
def 夏普指數(df):
    # 計算日報酬率
    df["日報酬率"] = df["收盤價"].pct_change()
    日報酬率 = df["日報酬率"].dropna().tolist()
    
    # 步驟 1：平均日報酬
    n = len(日報酬率)
    平均日報酬 = sum(日報酬率) / n
    
    # 步驟 2：樣本標準差（ddof = 1）
    平方差 = [(x - 平均日報酬) ** 2 for x in 日報酬率]
    樣本日標準差 = (sum(平方差) / (n - 1))**0.5
    
    # 步驟 3：年化報酬率
    年化報酬率 = (df["收盤價"].iat[-1]/df["收盤價"][0])-1
    
    # 步驟 4：年化標準差（波動度）
    年化標準差 = 樣本日標準差 * (n ** 0.5)
    
    # 步驟 5：無風險利率
    無風險利率 = 0.01
    
    # 步驟 6：夏普比率
    夏普比率 = (年化報酬率 - 無風險利率) / 年化標準差
    return 夏普比率

# 指定查詢時間（雖只抓取年月，但日一樣要填）
年份 = 2024
月份 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(1, 13)]

股票代碼=["0050","0051","0055","0056"]
比較表={"股票代碼":[],"夏普指數":[]}
for st in 股票代碼:
    data=下載股價資訊(月份,st)
    夏普=夏普指數(data)
    比較表["股票代碼"].append(st)
    比較表["夏普指數"].append(夏普)
pd.DataFrame(比較表)

In [ ]:
import requests
import pandas as pd
re=requests.get("https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20250601&stockNo=2330")
re=re.json()
df=pd.DataFrame(re["data"],columns=re["fields"])
def 民國轉西元(date_str):
    parts = date_str.split('/')
    parts[0] = str(int(parts[0]) + 1911)
    return '/'.join(parts)
print(df)
df["日期"] = df["日期"].apply(民國轉西元)
df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m-/%d")
# df

In [ ]:
import requests
import pandas as pd
import math
import time
def 民國轉西元(date_str):
    parts = date_str.split('/')
    parts[0] = str(int(parts[0]) + 1911)
    return '/'.join(parts)
def 下載股價資訊(月份列表,股票代碼):
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        df["日期"] = df["日期"].apply(民國轉西元)
        df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
        
        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)


# 指定查詢時間（雖只抓取年月，但日一樣要填）
年份 = 2024
月份 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(1, 13)]

# 下載資料
股票代碼 = "2330"
df = 下載股價資訊(月份,股票代碼)
df

In [ ]:
# 投資資料
買入價格 = 100
賣出價格 = 140
股利收入 = 20
持有年數 = 4 

# 計算總報酬率
報酬率 = (賣出價格 - 買入價格 + 股利收入) / 買入價格

# 計算年化報酬率
年化報酬率 = (1 + 報酬率) ** (1 / 持有年數) - 1

print(f"報酬率: {報酬率:.2%}")
print(f"年化報酬率: {年化報酬率:.2%}")

In [ ]:
"""
使用twse API抓取上市個股日成交資訊並計算夏普指數
https://openapi.twse.com.tw/#/%E8%AD%89%E5%88%B8%E4%BA%A4%E6%98%93/get_exchangeReport_STOCK_DAY_ALL
"""
import requests
import pandas as pd
import time

def 下載股價資訊(月份列表,股票代碼):
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            print(f"{date_str[0:6]}抓取失敗")
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
    
def 民國轉西元(date_str):
    parts = date_str.split('/')
    parts[0] = str(int(parts[0]) + 1911)
    return '/'.join(parts)

# 指定查詢時間（雖只抓取年月，但日一樣要填）
年份 = 2024
月份 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(1, 13)]

# 下載資料
股票代碼 = ["0050","0051","0053","0055","0056"]
def 股票指標試算(stock):
    df = 下載股價資訊(月份,stock)
    
    # 處理日期：轉為西元年
    df["日期"] = df["日期"].apply(民國轉西元)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 計算日報酬率
    df["日報酬率"] = df["收盤價"].pct_change()
    日報酬率 = df["日報酬率"].dropna().tolist()
    
    # 步驟 1：平均日報酬
    n = len(日報酬率)
    平均日報酬 = sum(日報酬率) / n
    
    # 步驟 2：樣本標準差（ddof = 1）
    平方差 = [(x - 平均日報酬) ** 2 for x in 日報酬率]
    樣本日標準差 = (sum(平方差) / (n - 1))**0.5
    
    # 步驟 3：年化報酬率
    年化報酬率 = (df["收盤價"].iat[-1]/df["收盤價"][0])-1
    
    # 步驟 4：年化標準差（波動度）
    年化標準差 = 樣本日標準差 * (n ** 0.5)
    
    # 步驟 5：無風險利率
    無風險利率 = 0.01
    
    # 步驟 6：夏普比率
    夏普比率 = (年化報酬率 - 無風險利率) / 年化標準差
    print(f"{stock}的夏普比率 : {夏普比率:.2f}")
股票指標試算("0050")
# 結果輸出
print(df.head())
print(df.tail())
print(f"資料期間：{df['日期'].min().date()} 至 {df['日期'].max().date()}")
print(f"交易日數：{n}")
print(f"樣本日標準差：{樣本日標準差:.4f}")
print(f"平均日報酬率：{平均日報酬:.2%}")
print(f"年化報酬率：{年化報酬率:.2%}")
print(f"年化標準差：{年化標準差:.2%}")
print(f"夏普比率：{夏普比率:.2f}")

In [ ]:
import pandas as pd

# 假設一組日報酬率資料（單位：比例）
日報酬率 = pd.Series([0.01, 0.015, -0.005, 0.02, 0.005, -0.01])

# 計算平均日報酬率
平均日報酬率 = 日報酬率.mean()

# 年化報酬率（252 個交易日）
年化報酬率 = 平均日報酬率 * 252

# 設定目標報酬率 T（Sortino 分母基準），通常為 0 或 無風險利率
目標報酬率 = 0.0

# 計算每筆下行差異平方
下行差異平方 = [(min(r - 目標報酬率, 0))**2 for r in 日報酬率]

# 樣本下行標準差（ddof = 1）
n = len(日報酬率)
樣本下行標準差 = (sum(下行差異平方) / (n - 1))**0.5

# 年化下行標準差
年化下行標準差 = 樣本下行標準差 * (252**0.5)

# 設定無風險利率（年化）
無風險利率 = 0.01

# 計算索提諾比率
索提諾比率 = (年化報酬率 - 無風險利率) / 年化下行標準差

# 顯示結果
print(f"平均日報酬率：{平均日報酬率:.2%}")
print(f"年化報酬率：{年化報酬率:.2%}")
print(f"樣本下行標準差（日）：{樣本下行標準差:.6f}")
print(f"年化下行標準差：{年化下行標準差:.2%}")
print(f"索提諾比率：{索提諾比率:.2f}")

In [ ]:
start=int(input())
end=int(input())
年分 = [str(y) for y in range(start, end+1)]
# 月份 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(9, 13)]
# 月份
年分

In [ ]:
"""
使用twse API抓取上市個股日成交資訊並計算索提諾比率
https://openapi.twse.com.tw/#/%E8%AD%89%E5%88%B8%E4%BA%A4%E6%98%93/get_exchangeReport_STOCK_DAY_ALL
"""
import requests
import pandas as pd
import time

def 下載股價資訊(月份列表,股票代碼):
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            print(f"{date_str[0:6]}抓取失敗")
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
    
def 民國轉西元(date_str):
    parts = date_str.split('/')
    parts[0] = str(int(parts[0]) + 1911)
    return '/'.join(parts)

# 指定查詢時間（雖只抓取年月，但日一樣要填）
年份 = "2024"
月份 = [f"{年份}{str(m).zfill(2)}01" for m in range(1, 13)]

# 下載資料
股票代碼 = "2330"
df = 下載股價資訊(月份,股票代碼)

# 處理日期：轉為西元年
df["日期"] = df["日期"].apply(民國轉西元)
df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")

# 步驟 1計算日報酬率
df["日報酬率"] = df["收盤價"].pct_change()
日報酬率 = df["日報酬率"].dropna().tolist()

# 步驟 2：年化報酬率
年化報酬率 = (df["收盤價"].iat[-1]/df["收盤價"][0])-1

# 步驟 3：設定目標報酬率 T（Sortino 分母基準），通常為 0 或 無風險利率
目標報酬率 = 0.0

# 步驟 4：計算每筆下行差異平方
下行差異平方 = [(min(r - 目標報酬率, 0))**2 for r in 日報酬率]

# 步驟 5：下行標準差（ddof = 1）
n = len(日報酬率)
下行日標準差 = (sum(下行差異平方) / (n - 1))**0.5

# 步驟 6：年化標準差（波動度）
下行年化標準差 = 下行日標準差 * (n ** 0.5)

# 步驟 7：無風險利率
無風險利率 = 0.01

# 步驟 9：索比率
索提諾比率 = (年化報酬率 - 無風險利率) / 下行年化標準差

# 結果輸出
print(df.head())
print(df.tail())
print(f"資料期間：{df['日期'].min().date()} 至 {df['日期'].max().date()}")
print(f"交易日數：{n}")
print(f"下行日標準差：{下行日標準差:.4f}")
print(f"平均日報酬率：{平均日報酬:.2%}")
print(f"年化報酬率：{年化報酬率:.2%}")
print(f"下行年化標準差：{下行年化標準差:.2%}")
print(f"索提諾比率：{索提諾比率:.2f}")

In [ ]:
收盤價 = [100, 105, 102, 108, 103, 97, 99, 94, 96, 92]

歷史最高點 = 收盤價[0]
所有回撤 = []

for 價格 in 收盤價:
    if 價格 > 歷史最高點:
        歷史最高點 = 價格
    回撤 = (價格 - 歷史最高點) / 歷史最高點
    所有回撤.append(回撤)
    
最大回撤 = min(所有回撤)
print("最大回撤：", round(最大回撤, 4))

In [ ]:
((df["收盤價"]-df["收盤價"].cummax())/df["收盤價"].cummax()).min()

In [ ]:
import pandas as pd
# 假設的歷史日報酬率 (%)
returns = [
    0.009934, -0.002765, 0.012954, 0.030461, -0.004683,
    -0.004683, 0.031584, 0.016349, -0.003248, -0.003766,
    -0.001765, 0.023210, 0.009638, -0.004414, 0.024123,
    -0.001950, -0.001450, 0.002767, -0.004382, -0.003784,
    0.013705, -0.006527, 0.002790, -0.002268, 0.003937,
    0.009556, -0.006512, 0.016856, -0.004757, -0.005757,
    0.003943, 0.003288, 0.007718, -0.006252, 0.009247,
    -0.007081, 0.003533, 0.003134, -0.002071, 0.001161,
    -0.002995, 0.003982, -0.006899, -0.003837, -0.001255,
    -0.006184, -0.000775, -0.007201, 0.002752, -0.000185,
    -0.002106, -0.008370, -0.006677, 0.004771, -0.000265,
    -0.005443, -0.005033, 0.007039, -0.005832, 0.006692,
    -0.002204, -0.001623, -0.001360, -0.000133, -0.002152,
    0.002081, -0.006863, 0.001327, -0.007390, 0.010108,
    0.001105, -0.006058, -0.001959, -0.002739, 0.004769,
    0.000267, -0.007257, -0.001074, -0.005640, -0.002739,
    0.004918, -0.000249, 0.002734, -0.003756, 0.001938,
    -0.003216, -0.002765, 0.000402, 0.001196, 0.003691,
    0.003260, -0.002558, -0.000536, 0.002820, -0.002482,
    0.004995, 0.002187, 0.004949, -0.003286, 0.002618
]
日報酬率 = pd.Series(returns)

# 步驟 1：將報酬率遞增排序
日報酬率 = 日報酬率.sort_values()

# 步驟 2：取出第 5% 及 1% 分位數
VaR_005 = 日報酬率.iat[int(len(日報酬率)*0.05-1)]
VaR_001 = 日報酬率.iat[int(len(日報酬率)*0.01-1)]

# 輸出結果
print("VaR_0.05 =",VaR_005)
print("VaR_0.01 =",VaR_001)

In [ ]:
日報酬率.nsmallest(int(len(日報酬率)*0.05)).mean()

In [ ]:
import pandas as pd
# 假設的歷史日報酬率 (%)
returns = [
    0.009934, -0.002765, 0.012954, 0.030461, -0.004683,
    -0.004683, 0.031584, 0.016349, -0.003248, -0.003766,
    -0.001765, 0.023210, 0.009638, -0.004414, 0.024123,
    -0.001950, -0.001450, 0.002767, -0.004382, -0.003784,
    0.013705, -0.006527, 0.002790, -0.002268, 0.003937,
    0.009556, -0.006512, 0.016856, -0.004757, -0.005757,
    0.003943, 0.003288, 0.007718, -0.006252, 0.009247,
    -0.007081, 0.003533, 0.003134, -0.002071, 0.001161,
    -0.002995, 0.003982, -0.006899, -0.003837, -0.001255,
    -0.006184, -0.000775, -0.007201, 0.002752, -0.000185,
    -0.002106, -0.008370, -0.006677, 0.004771, -0.000265,
    -0.005443, -0.005033, 0.007039, -0.005832, 0.006692,
    -0.002204, -0.001623, -0.001360, -0.000133, -0.002152,
    0.002081, -0.006863, 0.001327, -0.007390, 0.010108,
    0.001105, -0.006058, -0.001959, -0.002739, 0.004769,
    0.000267, -0.007257, -0.001074, -0.005640, -0.002739,
    0.004918, -0.000249, 0.002734, -0.003756, 0.001938,
    -0.003216, -0.002765, 0.000402, 0.001196, 0.003691,
    0.003260, -0.002558, -0.000536, 0.002820, -0.002482,
    0.004995, 0.002187, 0.004949, -0.003286, 0.002618
]
日報酬率 = pd.Series(returns)

# 步驟 1：將報酬率遞增排序
日報酬率 = 日報酬率.sort_values()

# 步驟 2：取出第 5% 及 1% 分位數並計算平均
CVaR_005 = 日報酬率.nsmallest(int(len(日報酬率)*0.05)).mean()
CVaR_001 = 日報酬率.nsmallest(int(len(日報酬率)*0.01)).mean()

# 輸出結果
print("CVaR_0.05 =",CVaR_005)
print("CVaR_0.01 =",CVaR_001)

In [ ]:
0050 51 53 55 56